In [1]:
from kinetic_measure import *
import plot_property_map as ppm
import pynbody
import numpy as np
import scipy.optimize as so
import scipy.stats as ss
import os


In [2]:
savedir = '/home/seoyoung/analysis/figure/'

simdir = '/project/rrg-babul-ad/rennehan/romulus_data/h1.cosmo50/'
param_file = '/project/rrg-babul-ad/rennehan/romulus_data/h1.cosmo50/h1.cosmo50PLK.1536gst1bwK1BH.param'

filelist = os.listdir(simdir)

In [3]:
result = [i[-6:] for i in filelist if i.startswith('h1.cosmo50') and len(i)==35]
result.sort()

In [8]:
snap = result[0]
data_file = '/project/rrg-babul-ad/rennehan/romulus_data/h1.cosmo50/h1.cosmo50PLK.1536gst1bwK1BH.'+snap
s = pynbody.load(data_file, paramfile = param_file)
s.physical_units()

In [9]:
s.properties['z']

12.878281506560166

In [5]:
def get_2dimage(s,filt_size, re_cal):
    s_filt = s[pynbody.filt.Sphere(str(filt_size)+' kpc h^-1')]

    def for_one_LOS(s_filt,i):
        print("get rgb data")
        rgb = get_rgb(s_filt, filt_size/s_filt.properties['h'], nbin,add_info = "_random"+str(i)+"_z_"+zred, re_cal = re_cal)
        
        spin_axis_tot = get_spin_axis(s_filt.star)

        print("get IFS data")
        v_spax, sig_spax = get_IFS(s_filt.star,half_size = filt_size/s_filt.properties['h'], nbin = nbin, add_info = "_random"+str(i)+"_z_"+zred, re_cal = re_cal)

        plt.subplot(3,4,i*4-3)
        print("get formation time plot")
        ppm.plot(s_filt.star['pos'][:,0],s_filt.star['pos'][:,1], s_filt.star['tform'], npix = nbin, vmax = 6.5,vmin = 2, cmap = 'jet', log = False, cbar = True)
        plt.contour(rgb, colors = 'k', extent = [-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h'],-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h']])

        plt.subplot(3,4,i*4-2)
        print("get metalicity plot")
        ppm.plot(s_filt.star['pos'][:,0],s_filt.star['pos'][:,1], s_filt.star['feh'], vmax = 0.01, vmin = -1,npix = nbin, cmap = 'jet', log = False, cbar = True)
        plt.contour(rgb, colors = 'k', extent = [-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h'],-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h']])


        plt.subplot(3,4,i*4-1)
        plt.imshow(v_spax.T[::-1], cmap = 'jet', vmax = 500, vmin = -500,  origin='upper', extent = [-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h']+2*filt_size/s_filt.properties['h']/nbin,-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h']+2*filt_size/s_filt.properties['h']/nbin])
        plt.colorbar()

        plt.contour(rgb, colors = 'k', extent = [-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h'],-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h']])

        plt.subplot(3,4,4*i)
        plt.imshow(sig_spax.T[::-1], cmap = 'jet',vmax = 500, vmin = 0,  origin='upper', extent = [-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h']+2*filt_size/s_filt.properties['h']/nbin,-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h']+2*filt_size/s_filt.properties['h']/nbin])
        plt.colorbar()

        plt.contour(rgb, colors = 'k', extent = [-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h'],-filt_size/s_filt.properties['h'],filt_size/s_filt.properties['h']])
    

    nbin = 100
    zred = str(round(s_filt.properties['z'],3))
    print(zred)
    
    
    plt.figure(figsize = (24,15))
    plt.suptitle("z = "+zred, fontsize = 30)
    
    print("First LOS")
    for_one_LOS(s_filt,1)
    
    s_filt.rotate_x(90)
    print("Second LOS")
    for_one_LOS(s_filt,2)

    s_filt.rotate_y(90)
    print("Third LOS")
    for_one_LOS(s_filt,3)


    plt.savefig(savedir+"2dimage_z_"+str(zred)+".png", bbox_inches = 'tight')
    
    plt.clf()

In [6]:
def one_snapshot(snap,nbin, re_cal):
    data_file = '/project/rrg-babul-ad/rennehan/romulus_data/h1.cosmo50/h1.cosmo50PLK.1536gst1bwK1BH.'+snap
    s = pynbody.load(data_file, paramfile = param_file)
    s.physical_units()
    
    print("get halos")
    h = s.halos()
    h0 = h[0]
    pynbody.analysis.halo.center(h0,mode='hyb')
    
    get_2dimage(s,nbin, re_cal) # [kpc/h] unit


In [ ]:
for i in range(0,len(result),1):
    try:
        snap = result[::-1][i]
        one_snapshot(snap, 35, re_cal = False) #50kpc at redshift 0 = 35kpc/h
    except: pass

get halos
2.028
First LOS
get rgb data
h2 information found in StarLog!
h2 information found in StarLog!
h2 information found in StarLog!


/home/seoyoung/anaconda3/lib/python3.7/site-packages/pynbody/plot/stars.py:74: RuntimeWarning: divide by zero encountered in log10
  return -2.5*np.log10(image*pc2_to_sqarcsec)


get IFS data
caluculate binned statistic


/home/seoyoung/anaconda3/lib/python3.7/site-packages/pynbody/array.py:865: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return self.base[self._reexpress_index(item)]
/home/seoyoung/anaconda3/lib/python3.7/site-packages/scipy/stats/_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]
/home/seoyoung/anaconda3/lib/python3.7/site-packages/scipy/stats/_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[se

get formation time plot


...

END PLOT
get metalicity plot


/home/seoyoung/anaconda3/lib/python3.7/site-packages/pynbody/array.py:865: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return self.base[self._reexpress_index(item)]


START PLOT


/home/seoyoung/anaconda3/lib/python3.7/site-packages/scipy/stats/_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]


END PLOT
get halos
